# Dependencies

In [2]:
from PIL import Image, ImageOps
import pylab
import hashlib
import progressbar
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from random import randint

from scipy import signal
from scipy.interpolate import griddata
from sklearn.decomposition import FastICA
from sklearn.feature_extraction import image as skimage
from ipywidgets import interact, interactive, fixed
%matplotlib inline

# Utility Functions

In [3]:
def generate_gabor(size, shift, sigma, rotation, phase_shift, frequency):
    radius = (int((size[0]/2.0)), int((size[1]/2.0)))
    [x, y] = np.meshgrid(range(-radius[0], radius[0]), range(-radius[1], radius[1])) # a BUG is fixed in this line
    x = x - int(shift[0])
    y = y - int(shift[1])
    x = x * frequency
    y = y * frequency
    tmp = x * np.cos(rotation) + y * np.sin(rotation) + phase_shift
    radius = (int(size[0]/2.0), int(size[1]/2.0))
    [x, y] = np.meshgrid(range(-radius[0], radius[0]), range(-radius[1], radius[1])) # a BUG is fixed in this line
    
    x = x - int(shift[0])
    y = y - int(shift[1])
    x1 = x * np.cos(rotation) + y * np.sin(rotation)
    y1 = -x * np.sin(rotation) + y * np.cos(rotation)
    
    sinusoid = np.cos(tmp)
    
    gauss = np.e * np.exp(np.negative(0.5 * ((x1**2 / sigma[0]**2) + (y1**2 / sigma[1]**2)))) 
    gauss = gauss / 2*np.pi * sigma[0] * sigma[1]
    
    gabor = gauss * sinusoid
    return gabor

def open_norm(path,verbose=False):
    raw = np.array(Image.open(path).convert("L"))
    norm = (raw - np.mean(raw)) / np.std(raw)
    
    if verbose:
        return raw, norm
    else:
        return norm
    
def linear_convolution(center, slide): 
    if (center.shape != slide.shape):
        return
    padded_slide = np.zeros((center.shape[0],center.shape[1]*3))
    padded_slide[0:,center.shape[1]:center.shape[1]*2] = center
    #plt.imshow(padded_slide,origin="lower")
    #plt.show()
    estimate = np.zeros([center.shape[1]*2])
    for x in range(center.shape[1]*2):
        dot = np.sum(padded_slide[0:,0+x:center.shape[1]+x] * slide)
        estimate[x] = dot
    #plt.plot(estimate)
    #plt.show()
    return np.abs(estimate)
    
def double_convolve(normal, shifted, image, pupillary_distance):
    
    #CHECKOUT https://github.com/maweigert/gputools
    #probably VERY advantageous to switch over to GPU for convolutions!
    
    normal_convolved = signal.convolve2d(image, normal, boundary='symm', mode='same')
    shifted_convolved = signal.convolve2d(image, shifted, boundary='symm', mode='same')
    
    return_shape = image.shape
    
    realigned = np.zeros(return_shape)

    


    normal_convolved = normal_convolved[0:,0:-pupillary_distance]
    shifted_convolved = shifted_convolved[0:,pupillary_distance:]
    

    
    
    diff = np.subtract(normal_convolved, shifted_convolved)
    mul = normal_convolved * shifted_convolved
    #plt.imshow(mul,cmap="nipy_spectral")
    #plt.show()
    
    #REMOVE BELOW COMMENTS TO THRESH SUBHALF VALUES
    #low_values_flags = mul <= 0 #mul.max()*0.5  # Where values are low
    #mul[low_values_flags] = 0  # All low values set to 0
    realigned[0:,pupillary_distance:] = mul
    return np.abs(mul)

def scale_disparity(activity_map, disparity_map):
    scaled_disparity = np.zeros([activity_map.shape[0],activity_map.shape[1],disparity_map.shape[0]])
    scaled_disparity[:,:] = disparity_map
    for x in range(activity_map.shape[0]):
        for y in range(activity_map.shape[1]):
            scaled_disparity[x,y] = activity_map[x,y] * scaled_disparity[x,y]
            
    return scaled_disparity

# LGN Model

In [4]:
import random
def distance(x0, y0, x1, y1):
  return np.sqrt(pow(x0-x1,2) + pow(y0-y1,2))

class LGN: 
  """
  this class defines a model which generates binocular spontaneous activity
  """
  
  def __init__(self, width = 128, p = 0.5, r = 1.0, t = 1, trans = 0.0,
    make_wave = True, num_layers=2, random_seed=0):
    random.seed(random_seed)
    self.width = width
    self.p = p
    self.r = r
    self.t = t
    self.trans = trans
    self.num_layers = num_layers
    if make_wave:
      self.reset_wave()

  def reset_wave(self):
    """ create another random wave """
    # setting up the network
    w = self.width
    self.recruitable = np.random.rand(self.num_layers, w, w) < self.p
    self.tot_recruitable = len(np.where(self.recruitable)[0])
    self.tot_recruitable_active = 0
    self.tot_active = 0
    self.active = np.zeros([self.num_layers,w,w],bool)
    self.active_neighbors = np.zeros([self.num_layers,w,w],int)
    self.activated = []; # the recently active nodes
    
    if self.tot_recruitable > 0:
      while self.fraction_active() < 0.2:
        self.activate()  

  def fraction_active(self):
    """ returns the fraction of potentially recruitable cells which are active """
    if self.tot_recruitable > 0:
      return float(self.tot_recruitable_active) / self.tot_recruitable
    else:
      return nan

  def propagate(self):
    """ propagate the activity if a valid node has been activated """
    # activated only has recruitable and currently inactive members
    while len(self.activated) > 0:
      act_l, act_x, act_y = self.activated.pop()
      self.active[act_l,act_x,act_y] = True
      self.tot_active += 1
      self.tot_recruitable_active += 1
      for l in range(self.num_layers):
        for x in range(int(act_x-self.r),int(act_x+self.r+1)):
          for y in range(int(act_y-self.r),int(act_y+self.r+1)):
            if distance(act_x,act_y,x,y) <= self.r:
              xi = x % self.width
              yi = y % self.width
              if l != act_l: # spread the activity across layers
                if np.random.rand() < self.trans: # transfer the activity
                  self.active_neighbors[l, xi,yi] += 1
              else: # if it is the same layer
                self.active_neighbors[l, xi,yi] += 1
              if self.active_neighbors[l, xi,yi] == self.t and \
                not self.active[l, xi,yi]:
                if self.recruitable[l, xi,yi]:
                  self.activated.append([l, xi,yi])
                else: # activate the node but don't propagate the activity
                  self.active[l,xi,yi] = True
                  self.tot_active += 1

  def activate(self):
    """ activate a random potentially active node """
    if self.fraction_active() > 0.95:
      return
      
    # pick a random point
    while True:
      l = np.random.randint(0,self.num_layers)
      x = np.random.randint(0,self.width)
      y = np. random.randint(0,self.width)
      if (self.recruitable[l,x,y] and not self.active[l,x,y]):
        break
    self.activated.append([l,x,y])
    self.propagate()

  def correlation(self):
    """ returns the correlation between the left and right images """
    # the total number of activations in common
    # same_count = len(where(self.active[0,:,:] == self.active[1,:,:])[0])
    # return float(same_count) / (self.width * self.width)
    
    # create an activity matrix of 0's and 1's (instead of True and False)
    if self.num_layers < 2:
      print("monocular models cannot have correlations between eye layers")
      return 0
    w = self.width
    active01 = np.zeros([2,w,w],int)
    active01[where(self.active)] = 1
    
    mean0 = active01[0,:,:].mean()
    mean1 = active01[1,:,:].mean()
    std0 = active01[0,:,:].std()
    std1 = active01[1,:,:].std()
    cov = ((active01[0,:,:] - mean0) * (active01[1,:,:] - mean1)).mean()
    return cov / (std0 * std1)
    
  def make_img_mat(self, show_img=True):
    """ return a matrix of 1's and 0's showing the activity in both layers """
    img_array = np.zeros([self.num_layers,self.width,self.width])
    border_width = 10 if self.num_layers > 1 else 0
    w = self.width
    for l in range(self.num_layers):
        img = np.zeros([w, w], float)
        for x in range(0,w-1):
            for y in range(0,w-1):
                if self.active[l,x,y]:
                    img[x,y] = 1           
    
        img_array[l] = img         
        #plt.imshow(img)
        #plt.show()
        
    return img_array

In [5]:
def generate_patches(num_patches, patch_size, lgn_width, lgn_p, lgn_r, lgn_t, lgn_a):
    half_comp = patch_size**2
    patch_count = 0

    while (patch_count < num_patches):
        L = LGN(width = lgn_width, p = lgn_p, r = lgn_r, t = lgn_t, trans = lgn_a, make_wave = True, num_layers=2, random_seed=randint(1,100))
        layer_activity = L.make_img_mat()
        patches_1 = np.array(skimage.extract_patches_2d(layer_activity[0], (patch_size, patch_size)))
        patches_2 = np.array(skimage.extract_patches_2d(layer_activity[1], (patch_size, patch_size)))
        reshaped_patches_1 = patches_1.reshape(-1, patches_1.shape[1]*patches_1.shape[1])
        reshaped_patches_2 = patches_2.reshape(-1, patches_2.shape[1]*patches_2.shape[1])
        composite_patches = np.concatenate((reshaped_patches_1,reshaped_patches_2),axis=1)
        blacklist = []        
        for x in range(composite_patches.shape[0]):
            if composite_patches[x][:half_comp].std() == 0.0 or composite_patches[x][half_comp:].std() == 0.0:
                blacklist.append(x)
        composite_patches = np.delete(composite_patches, np.array(blacklist), axis=0)
        if (patch_count == 0):
            patch_base = composite_patches
        else:
            patch_base = np.append(patch_base, composite_patches, axis=0)
        patch_count = patch_base.shape[0]

    return patch_base[:num_patches]

In [33]:
def perform_ica(num_components, patches):
    # Run ICA on all the patches and return generated components
    ica_instance = FastICA(n_components=num_components, random_state=1,max_iter=1000) # note, sensitive to n_components
    icafit = ica_instance.fit(patches)
    ica_components = icafit.components_
    return ica_components

In [7]:
def generate_filters(num_filters, num_components, num_patches, patch_size, lgn_width, lgn_p, lgn_r, lgn_t, lgn_a): 
    print("GENERATING FILTERS")
    bar = progressbar.ProgressBar(max_value=num_filters)
    filter_count = 0
    while (filter_count < num_filters):
        patches = generate_patches(num_patches, patch_size, lgn_width, lgn_p, lgn_r, lgn_t, lgn_a)
        filters = perform_ica(num_components, patches)
        if (filter_count == 0):
            filter_base = filters
        else:
            filter_base = np.append(filter_base, filters, axis=0)
        filter_count = filter_base.shape[0]
        if (filter_count < num_filters):
            bar.update(filter_count)
        else:
            bar.update(num_filters)

    return filter_base[:num_filters]

In [8]:
def unpack_filters(filters):
    half_filter = int(filters.shape[1]/2)
    filter_dim = int(np.sqrt(filters.shape[1]/2))
    first_eye = filters[:, 0:half_filter].reshape(-1,filter_dim,filter_dim)
    second_eye = filters[:, half_filter:].reshape(-1,filter_dim,filter_dim)
    return (first_eye, second_eye)

In [9]:
def linear_disparity(first_eye, second_eye):
    disparity_map = np.empty([first_eye.shape[0],first_eye.shape[1]*2])
    for index in range(first_eye.shape[0]):
        disparity = linear_convolution(first_eye[index], second_eye[index])
        disparity_map[index] = disparity
    return disparity_map

In [21]:
def normalize_disparity(disparity_map):
    sum_disparity = np.sum(disparity_map, axis=0)
    with np.errstate(divide='ignore', invalid='ignore'):
        normalized_disparity = disparity_map / sum_disparity
    return normalized_disparity

In [27]:
def generate_activity(autostereogram, asg_patch_size, first_eye, second_eye, disparity_map):
    print("CALCULATING ACTIVITY")
    bar = progressbar.ProgressBar(max_value=first_eye.shape[0])
    for index in range(first_eye.shape[0]):
        #make this more elegant
        convolution = double_convolve(first_eye[index], second_eye[index], autostereogram, asg_patch_size)
        scaled_activity = scale_disparity(convolution,disparity_map[index])
        if index == 0:
            summed_activity = scaled_activity
        else:
            summed_activity = summed_activity + scaled_activity
        bar.update(index)
    bar.update(first_eye.shape[0])
    return summed_activity

In [12]:
def estimate_depth(activity):
    print("ESTIMATING DEPTH")
    depth_estimate = np.zeros([activity.shape[0],activity.shape[1]])
    bar = progressbar.ProgressBar(max_value=activity.shape[0])
    for x in range(activity.shape[0]):
        for y in range(activity.shape[1]):
            peak = np.abs(np.nanargmax(activity[x,y])-int(activity.shape[2]/2))
            peak = np.nanargmax(activity[x,y])
            depth_estimate[x,y] = peak
        bar.update(x)
    return depth_estimate

In [ ]:

test = generate_filters(1000, 5, 5000, 16, 128, 0.14, 3, 2, 0.2)
t = unpack_filters(test)
dm = linear_disparity(t[0],t[1])
nd = normalize_disparity(dm)


auto = open_norm("shift5_70patch_small.png",verbose=False)
asg_patch_sz = 70
act = generate_activity(auto, asg_patch_sz, t[0], t[1], nd)
de = estimate_depth(act)
plt.imshow(de)
plt.show()

GENERATING FILTERS


 46% (465 of 1000) |##########            | Elapsed Time: 0:02:59 ETA:  0:03:08

In [23]:
def runEx(pShift=0.0, R=3, Trans = 0.1):
    
    tarray = []
    parray = []
    carray = []
    harray = []
    
    patch_size = 8
    
    for bigT in range(1,5):
        bigP = bigT / (((np.pi * (3**2)/2))*1.1) + pShift
        print("t: ", bigT)
        print("p: ", bigP)

        feg = []
        seg = []

        for gg in range(5):


            patches_1 = []
            patches_2 = []
            for n in range(5):
                seed = randint(1,100)
                # Generate the spontaneous activity patterns
                L = LGN(width = 64, p = bigP, r = R, t = bigT, trans = Trans, make_wave = True, num_layers=2, random_seed=seed)
                # Save patterns as images
                images = L.make_img_mat()

                # Generate 16x16 image patches for both left and right eye using a sliding window
                patches_1.append(image.extract_patches_2d(images[0], (patch_size, patch_size)))
                patches_2.append(image.extract_patches_2d(images[1], (patch_size, patch_size)))

            patches_1 = np.array(patches_1)
            patches_2 = np.array(patches_2)

            blacklist = []

            for x in range(0,patches_1.shape[0]):
                if patches_1[x].std() == 0.0:
                    blacklist.append(x)
                    continue
                        #add to black list

                if patches_2[x].std() == 0.0:
                    blacklist.append(x)
                    #add to black list

            blacklist = np.array(blacklist)
            if (blacklist.shape[0] != 0):
                patches_1 = np.delete(patches_1, blacklist, axis=0)
                patches_2 = np.delete(patches_2, blacklist, axis=0)
                    #removes boring patches (the patches with no activity!)

            patches_1 = patches_1.reshape((patches_1.shape[0]*patches_1.shape[1]),patch_size,patch_size)
            patches_2 = patches_2.reshape((patches_2.shape[0]*patches_2.shape[1]),patch_size,patch_size)

            # Reshape the array to the shape (patches, the size of the patches)
            reshaped_patches_1 = patches_1.reshape(-1, patches_1.shape[1]*patches_1.shape[1])
            reshaped_patches_2 = patches_2.reshape(-1, patches_2.shape[1]*patches_2.shape[1])
            composite_patches = np.concatenate((reshaped_patches_1,reshaped_patches_2),axis=1)

            # Number of components to create/use
            n_ica_components = 20

            # Run ICA on all the patches and return generated components
            icatemp = FastICA(n_components=n_ica_components, random_state=1) # note, sensitive to n_components
            icafit_1 = icatemp.fit(composite_patches)
            ica_comp_1 = icafit_1.components_

            # Get the components from both eye layers
            first_eye = ica_comp_1[:, 0:patches_1.shape[1]**2]
            second_eye = ica_comp_1[:, patches_1.shape[1]**2:]

            # Reshape components (for easier plotting)
            results_1 = first_eye.reshape(-1,patches_1.shape[1],patches_1.shape[1])
            results_2 = second_eye.reshape(-1,patches_1.shape[1],patches_1.shape[1])
            feg.append(results_1)
            seg.append(results_2)



        feg = np.array(feg)
        seg = np.array(seg)
        first_eye_gabors = np.array(feg).reshape((feg.shape[0] * feg.shape[1]),patch_size,patch_size)
        second_eye_gabors = np.array(seg).reshape((seg.shape[0] * seg.shape[1]),patch_size,patch_size)
        auto = open_norm("shift5_70patch.png",verbose=False)

        initi = True

        for x in range(0,first_eye_gabors.shape[0]):
            am1 = double_convolve(first_eye_gabors[x], second_eye_gabors[x], auto,70)
            dm1 = linear_convolution(first_eye_gabors[x], second_eye_gabors[x])
            if ((np.argmax(np.abs(dm1))-patch_size) != 0):
                sd1 = scale_disparity(am1,dm1)
                if initi == True:
                    ca = sd1
                    initi = False
                else:
                    ca = ca + sd1



        depth_estimate = np.zeros([ca.shape[0],ca.shape[1]])

        for x in range(ca.shape[0]):
            for y in range(ca.shape[1]):
                peak = np.abs(np.argmax(np.abs(ca[x,y]))-patch_size)
                depth_estimate[x,y] = peak




        dm = np.array(Image.open("dm.png").convert("L"))
        corr = np.corrcoef(depth_estimate[0:600,70:670].flatten(),dm[0:600,70:670].flatten())[0,1]
        print("corr: ", corr)
        


        hasher = "%f%f%f" % (bigT, bigP, corr)
        hasher = hashlib.sha256(hasher.encode('utf-8')).hexdigest()
        hasher = hasher[:20]
        print("hash: ", hasher)
        namer = "lgn_corr_data/%s.png" % (hasher)

        plt.imshow(depth_estimate[0:600,70:670], cmap="binary")
        plt.show()

        
        saver = depth_estimate[0:600,70:670]
        
        act = (255.0 / saver.max() * (saver - saver.min())).astype(np.uint8)
        sv = Image.fromarray(act)
        sv = ImageOps.colorize(sv, (0,0,0), (0,255,0))
        sv.save(namer)
        
        tarray.append(bigT)
        parray.append(bigP)
        carray.append(corr)
        harray.append(hasher)

        
    return {"t": tarray, "p": parray, "c": carray, "h": harray}


In [ ]:
zero = runEx(0.0)

plusp1 = runEx(0.01)
plusp2 = runEx(0.02)

minusp1 = runEx(-0.01)
minusp2 = runEx(-0.02)

In [ ]:
#CHANGE LINE 21

print(zero)
print()

print(plusp1)
print()

print(plusp2)
print()

print(minusp1)
print()

print(minusp2)
print()


In [ ]:
T = []
P = []
C = []

T.append(zero["t"])
T.append(plusp1["t"])
T.append(plusp2["t"])
T.append(minusp1["t"])
T.append(minusp2["t"])
T = np.array(T).flatten()

P.append(zero["p"])
P.append(plusp1["p"])
P.append(plusp2["p"])
P.append(minusp1["p"])
P.append(minusp2["p"])
P = np.array(P).flatten()

C.append(zero["c"])
C.append(plusp1["c"])
C.append(plusp2["c"])
C.append(minusp1["c"])
C.append(minusp2["c"])
C = np.array(C).flatten()


In [ ]:
print(C)

In [ ]:
p = np.column_stack((T,P))




xi = np.linspace(T.min(), T.max(), 100)
yi = np.linspace(P.min(), P.max(), 100)
xi, yi = np.meshgrid(xi, yi)

# interpolate
zi = griddata(p, C, (xi, yi), method="cubic")

plt.contourf(xi, yi, zi)

plt.title('FLAW: trans = 0.1, r = 3')


h = plt.ylabel('P   ')
h.set_rotation(0)
plt.xlabel('T')


plt.savefig("contour.png")

In [ ]:
'''

This cell intentionally left blank







{'t': [1, 2, 3, 4, 5, 6, 7, 8], 'p': [0.0643050275118769, 0.1286100550237538, 0.1929150825356307, 0.2572201100475076, 0.3215251375593845, 0.3858301650712614, 0.45013519258313833, 0.5144402200950152], 'c': [0.2271524872176735, 0.11748923387279109, 0.16814365075272392, 0.11891839158822315, 0.22924557006867541, 0.2021914390136689, 0.17564410008960482, 0.11591208048129203], 'h': ['2be4e854a3a8019f10ce', 'd7a4863ed4d821dd12da', '7574c16b165310f308f6', 'a29c1089edfd07b87cc4', 'd1cfe3053f6e64b2ae1f', 'c7cee8860f2a854d4979', '75012fbdb92846eac602', '054c4c7786a59a03ffc3']}

{'t': [1, 2, 3, 4, 5, 6, 7, 8], 'p': [0.16430502751187692, 0.2286100550237538, 0.2929150825356307, 0.35722011004750764, 0.4215251375593845, 0.4858301650712614, 0.5501351925831384, 0.6144402200950152], 'c': [0.17965148101740655, 0.18346965101910506, 0.19459529299072095, 0.076159217264992848, 0.15864535098727001, 0.13156357558363777, 0.14233424494430186, 0.22089215028050982], 'h': ['608afb5e7d3248462cbe', '9ccec377250821de4001', 'c7193270aa4c7ff3fe14', '237331c17e259f07b36d', '1d1fdd00cf923c6be634', '82ade1e5a50a29cb2e74', 'dff663d5d61841f782d4', '669ca9375b66d627319c']}

{'t': [1, 2, 3, 4, 5, 6, 7, 8], 'p': [0.2643050275118769, 0.32861005502375384, 0.3929150825356307, 0.4572201100475076, 0.5215251375593846, 0.5858301650712614, 0.6501351925831383, 0.7144402200950153], 'c': [0.10099254151515953, 0.20184900352887408, 0.13151115566586377, 0.11472869083386326, 0.15515506356514958, 0.082535541773616827, 0.20066737038524315, 0.18114673542172366], 'h': ['922a020aa7cbd348cd7d', 'd7be78f247bcc037902f', 'ab69fc8b784e6c505b09', '70f34a8b12410be24ab5', '20c7bae346f1bb34373f', '350037822f2e8440525d', '8b2b88bae00683e7a873', '2b76bb991235ecacc4c5']}

{'t': [1, 2, 3, 4, 5, 6, 7, 8], 'p': [-0.0356949724881231, 0.0286100550237538, 0.09291508253563069, 0.1572201100475076, 0.2215251375593845, 0.28583016507126136, 0.3501351925831383, 0.41444022009501524], 'c': [0.10247495422255036, 0.14378019690175797, 0.20006787356858022, 0.1611758406682812, 0.14768773606739419, 0.21104334560934948, 0.12958418492805665, 0.13361840455817844], 'h': ['a4a3af560d6846203328', 'a4cd5cacdbe45e034f43', 'bad2bdf032712b33681c', 'f52aee518e038af56d2a', '1d1392e1699c4c2c1fe3', '60568bc2f83ba088a340', 'ee2540a353631fdb3fac', '674586e1d14430f357e1']}

{'t': [1, 2, 3, 4, 5, 6, 7, 8], 'p': [-0.13569497248812312, -0.0713899449762462, -0.007084917464369317, 0.0572201100475076, 0.12152513755938449, 0.18583016507126138, 0.2501351925831383, 0.3144402200950152], 'c': [0.20084754601983387, 0.19575105078370753, 0.15312192862797003, 0.12826216298868348, 0.19207832092450744, 0.19047879777972482, 0.21390114541489133, 0.22404160860323402], 'h': ['a1b6cc26fb7a61d4e4c8', '7217e4ac82668cb964c3', 'e02d0dfcd0219c93d7e9', '990a591cfe25e5dafa12', '5f187492118f439c15cc', 'bf9fc67a782462dfe82c', '3dfb8bd11d355b51dd0e', 'b5ff87bce29f3fe272b0']}



t = [1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 8 1 2 3 4 5 6 7 8]
 
 p = [ 0.06430503  0.12861006  0.19291508  0.25722011  0.32152514  0.38583017
  0.45013519  0.51444022  0.16430503  0.22861006  0.29291508  0.35722011
  0.42152514  0.48583017  0.55013519  0.61444022  0.26430503  0.32861006
  0.39291508  0.45722011  0.52152514  0.58583017  0.65013519  0.71444022
 -0.03569497  0.02861006  0.09291508  0.15722011  0.22152514  0.28583017
  0.35013519  0.41444022 -0.13569497 -0.07138994 -0.00708492  0.05722011
  0.12152514  0.18583017  0.25013519  0.31444022]







c = [ 0.22715249  0.11748923  0.16814365  0.11891839  0.22924557  0.20219144
  0.1756441   0.11591208  0.17965148  0.18346965  0.19459529  0.07615922
  0.15864535  0.13156358  0.14233424  0.22089215  0.10099254  0.201849
  0.13151116  0.11472869  0.15515506  0.08253554  0.20066737  0.18114674
  0.10247495  0.1437802   0.20006787  0.16117584  0.14768774  0.21104335
  0.12958418  0.1336184   0.20084755  0.19575105  0.15312193  0.12826216
  0.19207832  0.1904788   0.21390115  0.22404161]









//////////////////////////////////// r2 //////////////////////////////////////////////////




{'t': [1, 2, 3], 'p': [0.0643050275118769, 0.1286100550237538, 0.1929150825356307], 'c': [0.1377634742857804, 0.14544935917455801, 0.15288387668259798], 'h': ['4bfaa1ecdc9434d1e6b1', '9169115640a7585626c5', '942e8421687c5501c75e']}

{'t': [1, 2, 3], 'p': [0.0743050275118769, 0.13861005502375381, 0.2029150825356307], 'c': [0.13410292614588934, 0.17328774382671466, 0.21582283877716074], 'h': ['d8a03faf2694f876d523', 'd80062e7c68bbbe9f5df', '361a1fbce3edaf05892c']}

{'t': [1, 2, 3], 'p': [0.0843050275118769, 0.1486100550237538, 0.21291508253563068], 'c': [0.19929353870166028, 0.21228907033779296, 0.21630368862593366], 'h': ['49bcf7142cf4bf871cfb', 'f55f2f77af41b127be25', '44399eb5405b349e5268']}

{'t': [1, 2, 3], 'p': [0.0543050275118769, 0.11861005502375381, 0.18291508253563069], 'c': [0.15707454586864894, 0.16889960493069478, 0.11172137044140341], 'h': ['6609816082c047a8e3de', 'f61c5d58deed5272fbc2', '9432d007955b2eff43d6']}

{'t': [1, 2, 3], 'p': [0.0443050275118769, 0.1086100550237538, 0.1729150825356307], 'c': [0.14943104383301722, 0.20341405899828638, 0.20558845806693285], 'h': ['9667779334d0c940d3f5', '921ea736648dd558fe9d', '6dd15e388993d5f4a66f']}



t = [1 2 3 1 2 3 1 2 3 1 2 3 1 2 3]

p = [ 0.06430503  0.12861006  0.19291508  0.07430503  0.13861006  0.20291508
  0.08430503  0.14861006  0.21291508  0.05430503  0.11861006  0.18291508
  0.04430503  0.10861006  0.17291508]
  
c = [ 0.13776347  0.14544936  0.15288388  0.13410293  0.17328774  0.21582284
  0.19929354  0.21228907  0.21630369  0.15707455  0.1688996   0.11172137
  0.14943104  0.20341406  0.20558846]



























'''

In [ ]:
feg = np.array(feg)
seg = np.array(seg)
first_eye_gabors = np.array(feg).reshape((feg.shape[0] * feg.shape[1]),16,16)
second_eye_gabors = np.array(seg).reshape((seg.shape[0] * seg.shape[1]),16,16)



plt.imshow(first_eye_gabors[499])
plt.show()


In [ ]:
sv = PIL.ImageOps.colorize(sv, (0,0,0), (0,255,0))


In [ ]:
plt.imshow(first_eye_gabors[0])
plt.show()

print(first_eye_gabors[0].shape)

auto = open_norm("shift5_70patch.png",verbose=False)

initi = True

for x in range(0,first_eye_gabors.shape[0]):
    am1 = double_convolve(first_eye_gabors[x], second_eye_gabors[x], auto,70)
    dm1 = linear_convolution(first_eye_gabors[x], second_eye_gabors[x])
    if ((np.argmax(np.abs(dm1))-16) != 0):
        sd1 = scale_disparity(am1,dm1)
        if initi == True:
            ca = sd1
            initi = False
        else:
            ca = ca + sd1
        


depth_estimate = np.zeros([ca.shape[0],ca.shape[1]])

for x in range(ca.shape[0]):
    for y in range(ca.shape[1]):
        peak = np.abs(np.argmax(np.abs(ca[x,y]))-16)
        depth_estimate[x,y] = peak
        
        
plt.imshow(depth_estimate[0:600,70:670], cmap="binary")
plt.show()
dm = np.array(PIL.Image.open("dm.png").convert("L"))
corr = np.corrcoef(depth_estimate[0:600,70:670].flatten(),dm[0:600,70:670].flatten())[0,1]
print(corr)


In [ ]:

        
        
plt.imshow(depth_estimate[0:600,70:670])
plt.savefig('foo.png', dpi = 300)
plt.show()





In [ ]:
am1 = double_convolve(first_eye_gabors[3], second_eye_gabors[3], auto,70)
plt.imshow(am1)
plt.show()

In [ ]:
for bigT in range(1,9):
    bigP = bigT / (((np.pi * (3**2)/2))*1.1)
    print(bigP)

In [ ]:
x = np.random.rand(1000, 2)
x.shape